# Homework 5 Supplemental Notebook

## DSC 40A, Winter 2023

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

## Problem 1 – What do you k-mean?

### Part B

**Note that you don't need to write any code yourself in this question.**

Run the cell below to load in our dataset, which contains many (209) attributes for each country, taken from [here](https://databank.worldbank.org/source/world-development-indicators#).

In [ ]:
world_bank = pd.read_csv('data/world_bank_data.csv') \
                .set_index('country') \
                .fillna(0) # There are null values in the dataset; .fillna(0) replaces them with 0s
world_bank

Let's try and cluster this dataset. Note that there's no easy way to visualize the dataset before clustering since it is 209-dimensional. We will use the KMeans method built into `scikit-learn`, a popular Python library for machine learning.

In [ ]:
from sklearn.cluster import KMeans

Run the cell below to perform the clustering and create a DataFrame with the results. (`random_state=42` ensures that we get the same results every time we run this cell.)

In [ ]:
world_bank_clustering = KMeans(n_clusters=4, random_state=42).fit(world_bank)
countries_and_labels = pd.DataFrame(index=world_bank.index) \
                         .assign(cluster=world_bank_clustering.labels_.astype(str))

In [ ]:
countries_and_labels

Now we have a cluster for each country, but it's kind of hard to tell which country is in which cluster. Run the cell below to see a map.

In [ ]:
px.choropleth(countries_and_labels.reset_index(),
              locations = 'country',
              locationmode = 'country names',
              color = 'cluster',
              hover_name = 'country',
              title = 'Country Clusters',
              category_orders = {'cluster': ['0', '1', '2', '3']}
)

If you aren't the greatest at geography, don't worry – you can hover over a country to see its name.

If you'd prefer, you can look at the cluster assignments in text form below.

In [ ]:
# Run this cell.
def show_all_clusters(cluster_df):
    for v in sorted(np.unique(cluster_df.get('cluster'))):
        print(f'Cluster {v}:', list(cluster_df[cluster_df.get('cluster') == v].get('cluster').index), '\n')
        
show_all_clusters(countries_and_labels)

You may notice an issue – the vast majority of countries are in the same cluster! That's not all that useful.

It turns out that if we **standardize** our data, as we do when computing standardized regression coefficients, the results turn out different. Run the cells below to standardize our dataset and re-run k-means clustering.

In [ ]:
world_bank_standardized = (world_bank - np.mean(world_bank, axis=0)) / np.std(world_bank, axis=0)
world_bank_standardized

In [ ]:
world_bank_standardized_clustering = KMeans(n_clusters=4, random_state=42).fit(world_bank_standardized)
countries_and_labels_standardized = pd.DataFrame(index=world_bank_standardized.index) \
                                      .assign(cluster=world_bank_standardized_clustering.labels_.astype(str))

We can draw the same map as before:

In [ ]:
px.choropleth(countries_and_labels_standardized.reset_index(),
              locations = 'country',
              locationmode = 'country names',
              color = 'cluster',
              hover_name = 'country',
              title = 'Country Clusters After Standardization',
              category_orders = {'cluster': ['0', '1', '2', '3']}
)

Note that in this new map, the clusters are a bit more "even" in terms of the number of countries in them. (The cluster numbers and colors are arbitrary, meaning that purple in the previous map is not related to purple in this map.)

In [ ]:
show_all_clusters(countries_and_labels_standardized)

<p style="color:red"><b>Your Job</b></p>

In your PDF writeup, write the answers to the following two questions:

(i) Come up with a simple description for each of the 4 clusters created after standardizing our features.

(ii) Why do you think standardizing our features before clustering had such an impact? (_Hint: Scroll up and compare the values in `world_bank` to those in `world_bank_standardized`.)